<a href="https://colab.research.google.com/github/LJMfintech/Financial_Data_Analysis_2/blob/main/DataAnalysisHW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload() # FTSE All-World Index 파일 업로드 하기

Saving FTSE All-World Index.csv to FTSE All-World Index.csv


In [ ]:
from google.colab import files
uploaded = files.upload() # dwcountreturns 파일 업로드 하기

Saving dwcountryreturns.csv to dwcountryreturns.csv


In [ ]:
from google.colab import files
uploaded = files.upload() # nsshock 올리기

Saving nsshock.csv to nsshock.csv


In [ ]:
"""HW2 Final Version (Weekend/Holiday Adjusted)."""

import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from datetime import timedelta

# ============================================================
# 0. FTSE All-World → WORLD return 불러오기
# ============================================================

df = pd.read_csv('FTSE All-World Index.csv')

def add_returns_to_df(df, price_col='prccd', return_col='return'):
    df[return_col] = df[price_col].pct_change()
    return df

df_subset = add_returns_to_df(df)
df_returns = df_subset[['datadate', 'return']]
df_returns.dropna(inplace=True)
df_returns['datadate'] = pd.to_datetime(df_returns['datadate'])
df_returns.to_csv('df_returns.csv', index=False)

# ============================================================
# 1. 국가별 수익률 + WORLD merge
# ============================================================

dwcountryreturns = pd.read_csv('dwcountryreturns.csv')
dwcountryreturns['datadate'] = pd.to_datetime(dwcountryreturns['datadate'].astype(str), format='%Y%m%d', errors='coerce')

df_returns = pd.read_csv('df_returns.csv')
df_returns['datadate'] = pd.to_datetime(df_returns['datadate'])

df_merged = pd.merge(df_returns, dwcountryreturns, on='datadate', how='inner')
df_merged_clean = df_merged.dropna()

df_merged_clean = df_merged_clean[['datadate','country','fic','n','currency','return','portret','portretx']]
df_merged_clean.to_csv('df_merged_clean.csv', index=False)

# ============================================================
# 2. NS shock 불러오기
# ============================================================

nsshock = pd.read_csv('nsshock.csv')
nsshock['datadate'] = pd.to_datetime(nsshock['datadate'])

nsshock = nsshock.sort_values("datadate").reset_index(drop=True)
nsshock["next_shock"] = nsshock["datadate"].shift(-1)

print("데이터 불러오기 완료")
print(df_merged_clean.head())
print(nsshock.head())

# ============================================================
# 3. 타임존 lag 정의
# ============================================================

country_time_lag = {
    'AUSTRALIA': 1, 'AUSTRIA': 1, 'BELGIUM': 1, 'BRAZIL': 0,
    'SWITZERLAND': 1, 'CHILE': 0, 'CHINA': 1, 'COLOMBIA': 0,
    'GERMANY': 1, 'DENMARK': 1, 'EGYPT': 1, 'SPAIN': 1,
    'FINLAND': 1, 'FRANCE': 1, 'UNITED KINGDOM': 1,
    'GREECE': 1, 'HONG KONG': 1, 'HUNGARY': 1, 'INDONESIA': 1,
    'INDIA': 1, 'IRELAND': 1, 'ITALY': 1, 'JAPAN': 1,
    'SOUTH KOREA': 1, 'MEXICO': 0, 'MALAYSIA': 1,
    'NETHERLANDS': 1, 'NORWAY': 1, 'NEW ZEALAND': 1,
    'PHILIPPINES': 1, 'POLAND': 1, 'PORTUGAL': 1,
    'SINGAPORE': 1, 'SWEDEN': 1, 'THAILAND': 1,
    'TURKEY': 1, 'TAIWAN': 1, 'SOUTH AFRICA': 1
}

# ============================================================
# 🔥 4. 주말·공휴일 고려된 거래일 추출 함수 (정상 버전)
# ============================================================

def get_next_trading_days(country_df, event_date, window=5):
    """
    shock 이벤트 이후 해당 국가에서 실제 발생한
    다음 window개 거래일을 반환한다.
    (주말·공휴일 자동 제외됨)
    """
    tmp = country_df[country_df['datadate'] >= event_date].sort_values("datadate")
    if len(tmp) < window:
        return None
    return tmp['datadate'].iloc[:window].tolist()


# ============================================================
# 5. 타임존 조정 + 이벤트 창 병합 함수 (최종 정답 버전)
# ============================================================

def merge_shock_with_time_lag(returns_df, shock_df, time_lag_map, event_window_days=5):

    merged_list = []

    for idx, shock_row in shock_df.iterrows():

        announcement = shock_row['datadate']        # shock date
        shock_val    = shock_row['NS']             # NS shock 값

        # -------------------------------------------------------
        # 이 부분이 핵심: next_shock cutoff 사용하지 않음
        # -------------------------------------------------------
        # → shock_i 이후 해당 국가의 실제 거래일만 기준으로 event window 생성
        # -------------------------------------------------------

        for country, lag in time_lag_map.items():

            # 특정 국가 데이터 가져오기
            country_data = returns_df[returns_df['country'] == country].copy()
            if len(country_data) == 0:
                continue

            # shock보다 국가 데이터가 늦게 시작되면 이벤트 불가능 → skip
            if country_data['datadate'].min() > announcement:
                continue

            # event_day 0 기준일 (타임존 lag 반영)
            event_date = announcement + timedelta(days=lag)

            # 🔥 next_shock 은 더 이상 사용하지 않음
            #    대신 해당 국가의 실제 거래일 기반 window 사용
            trading_days = get_next_trading_days(country_data, event_date, event_window_days)

            # 거래일 부족하면 skip
            if trading_days is None:
                continue

            # event_day 0 ~ 4 생성
            for event_day, tdate in enumerate(trading_days):
                row = country_data[country_data['datadate'] == tdate].iloc[0].copy()
                row["NS_shock"] = shock_val
                row["announcement_date"] = announcement
                row["event_day"] = event_day
                merged_list.append(row)

    return pd.DataFrame(merged_list)


# ============================================================
# 6. Problem 1 실행 (정상 작동)
# ============================================================

df_final = merge_shock_with_time_lag(df_merged_clean, nsshock, country_time_lag, event_window_days=5)

df_final = df_final.sort_values(['announcement_date','country','event_day']).reset_index(drop=True)
df_final.to_csv("problem1_final_merged_data.csv", index=False)

print("✓ Problem 1 완료")
print(df_final.head())

/tmp/ipython-input-914621726.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_returns.dropna(inplace=True)
/tmp/ipython-input-914621726.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_returns['datadate'] = pd.to_datetime(df_returns['datadate'])


데이터 불러오기 완료
    datadate    country  fic    n currency   return   portret  portretx
0 1986-07-01  AUSTRALIA  AUS   54      AUD  0.00549 -0.001796 -0.001796
1 1986-07-01    DENMARK  DNK   14      DKK  0.00549  0.012794  0.012794
2 1986-07-01  HONG KONG  HKG   16      HKD  0.00549 -0.000580 -0.000580
3 1986-07-01      JAPAN  JPN  638      JPY  0.00549 -0.003055 -0.003055
4 1986-07-01  SINGAPORE  SGP   17      SGD  0.00549  0.009316  0.009316
    datadate  GSS_target  GSS_path      NS next_shock
0 1995-02-01      0.0597    0.0268  0.0419 1995-03-28
1 1995-03-28      0.0258    0.0568  0.0288 1995-05-23
2 1995-05-23      0.0040   -0.0064  0.0009 1995-07-06
3 1995-07-06     -0.1033   -0.3785 -0.1510 1995-08-22
4 1995-08-22      0.0508    0.0217  0.0354 1995-09-26
✓ Problem 1 완료
    datadate    country  fic    n currency    return   portret  portretx  \
0 1995-02-02  AUSTRALIA  AUS  192      AUD  0.000942  0.010802  0.010802   
1 1995-02-03  AUSTRALIA  AUS  192      AUD  0.005058 -0.003422 -0

In [ ]:
# ============================================================
# Problem 2 — 시장모형 α,β 정상 추정 버전
# ============================================================

df_event = df_final.copy()

alpha = {}
beta = {}

countries = df_event['country'].unique()

for c in countries:

    # 이벤트 전 원시 데이터에서 추정해야 한다!
    raw_temp = df_merged_clean[df_merged_clean['country'] == c].copy()

    # 최소치 필터
    if len(raw_temp) < 50:
        continue

    # 시장모형: R_it = alpha + beta * R_mt
    X = sm.add_constant(raw_temp['return'])
    y = raw_temp['portret']

    model = sm.OLS(y, X, missing='drop').fit()
    alpha[c] = model.params['const']
    beta[c] = model.params['return']

# Expected return
df_event['ER'] = df_event.apply(
    lambda r: alpha.get(r['country'], np.nan) + beta.get(r['country'], np.nan) * r['return'],
    axis=1
)

# AR
df_event['AR'] = df_event['portret'] - df_event['ER']

# CAR
df_event['CAR'] = df_event.groupby(['announcement_date','country'])['AR'].cumsum()

df_event.to_csv("problem2_eventstudy_fixed.csv", index=False)

print("✓ Problem 2 정상 작동 (CAR 생성 완료)")


✓ Problem 2 정상 작동 (CAR 생성 완료)


In [ ]:
df_reg = df_event[df_event['event_day'] == 4].copy()

print(df_reg.shape)
print(df_reg[['CAR', 'NS_shock']].isna().sum())
print(df_reg[['CAR', 'NS_shock']].describe())

(8093, 14)
CAR         0
NS_shock    0
dtype: int64
               CAR     NS_shock
count  8093.000000  8093.000000
mean      0.001286     0.000066
std       0.024839     0.031283
min      -0.240141    -0.151000
25%      -0.010892    -0.009700
50%       0.001343     0.002800
75%       0.013181     0.013300
max       0.227647     0.093800


In [ ]:

# ============================================================
# 8. Problem 3 — 패널 회귀분석
# ============================================================

df_reg = df_event[df_event['event_day'] == 4].copy()

model = smf.ols(
    'CAR ~ NS_shock + C(country) + C(announcement_date)',
    data=df_reg
)
result = model.fit()

print(result.summary())

beta = result.params['NS_shock']
print(f"\nNS shock 계수 = {beta:.4f}")
print("✓ Problem 3 완료")

                            OLS Regression Results                            
Dep. Variable:                    CAR   R-squared:                       0.176
Model:                            OLS   Adj. R-squared:                  0.147
Method:                 Least Squares   F-statistic:                     6.179
Date:                Sat, 22 Nov 2025   Prob (F-statistic):          3.78e-176
Time:                        11:47:11   Log-Likelihood:                 19206.
No. Observations:                8093   AIC:                        -3.787e+04
Df Residuals:                    7822   BIC:                        -3.597e+04
Df Model:                         270                                         
Covariance Type:            nonrobust                                         
                                                               coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------

In [ ]:
# ===============================================================
# Full Summary Statistics Table
# ===============================================================

import pandas as pd

# 변수 정의
total_ret = df_event['portret']
car0 = df_event[df_event['event_day'] == 0]['CAR']
car1 = df_event[df_event['event_day'] == 1]['CAR']
car2 = df_event[df_event['event_day'] == 2]['CAR']
car3 = df_event[df_event['event_day'] == 3]['CAR']
car4 = df_event[df_event['event_day'] == 4]['CAR']
ns_shock = df_event['NS_shock']

# summary describe() 모두 포함
summary_table = pd.DataFrame({
    "Total Return": total_ret.describe(),
    "CAR(0)": car0.describe(),
    "CAR(1)": car1.describe(),
    "CAR(2)": car2.describe(),
    "CAR(3)": car3.describe(),
    "CAR(4)": car4.describe(),
    "NS Shock": ns_shock.describe()
})

# csv 저장
summary_table.to_csv("Table1_FullSummaryStatistics.csv")

summary_table

,Total Return,CAR(0),CAR(1),CAR(2),CAR(3),CAR(4),NS Shock
count,40465.000000,8093.000000,8093.000000,8093.000000,8093.000000,8093.000000,40465.000000
mean,0.000563,0.000884,0.001089,0.001022,0.000886,0.001286,0.000066
std,0.013130,0.011920,0.016497,0.020288,0.022734,0.024839,0.031281
min,-0.185470,-0.183134,-0.236727,-0.304080,-0.242601,-0.240141,-0.151000
25%,-0.005694,-0.005058,-0.007092,-0.008981,-0.010419,-0.010892,-0.009700
50%,0.000640,0.000721,0.000983,0.000454,0.000771,0.001343,0.002800
75%,0.006836,0.006552,0.008941,0.010575,0.011431,0.013181,0.013300
max,0.124006,0.102335,0.181732,0.187642,0.200185,0.227647,0.093800


In [ ]:
# ============================================================
# Problem 3 — Regression Table
# ============================================================
df_reg = df_event[df_event['event_day'] == 4].copy()

model = smf.ols(
    'CAR ~ NS_shock + C(country) + C(announcement_date)',
    data=df_reg
)
result = model.fit()

# === 회귀 테이블 생성 (변수명 포함) ===
reg_table = pd.DataFrame({
    "Variable": result.params.index,
    "Coefficient": result.params.values,
    "Std.Err": result.bse.values,
    "t-value": result.tvalues.values,
    "p-value": result.pvalues.values
})

# CSV 저장
reg_table.to_csv("Table2_RegressionResults.csv", index=False)

reg_table

,Variable,Coefficient,Std.Err,t-value,p-value
0,Intercept,0.009305,0.004209,2.210778,0.027080
1,C(country)[T.AUSTRIA],0.000076,0.002219,0.034161,0.972750
2,C(country)[T.BELGIUM],0.000561,0.002219,0.252831,0.800406
3,C(country)[T.BRAZIL],-0.000717,0.002128,-0.337040,0.736096
4,C(country)[T.CHILE],-0.001618,0.002291,-0.706131,0.480128
...,...,...,...,...,...
267,C(announcement_date)[T.Timestamp('2023-12-13 0...,0.004757,0.004517,1.053078,0.292338
268,C(announcement_date)[T.Timestamp('2024-01-31 0...,-0.013076,0.005850,-2.235136,0.025437
269,C(announcement_date)[T.Timestamp('2024-03-20 0...,0.000186,0.005256,0.035309,0.971834
270,C(announcement_date)[T.Timestamp('2024-05-01 0...,-0.011925,0.005406,-2.205837,0.027424
